# Import Library

In [1]:
from run import *
from tint.metrics import mse, mae
import tint, gc
from tqdm import tqdm
import pandas as pd
from utils.explainer import *
from utils.tsr_tunnel import TSRTunnel
from exp.exp_interpret import *

In [2]:
from captum.attr import (
    DeepLift,
    GradientShap,
    IntegratedGradients,
    Lime
)

from tint.attr import (
    AugmentedOcclusion,
    DynaMask,
    Occlusion, 
    FeatureAblation
)

# Argument Parser

In [3]:
parser = get_parser()
argv = """
  --task_name long_term_forecast \
  --use_gpu \
  --root_path ./dataset/illness/ \
  --data_path national_illness.csv \
  --model Autoformer \
  --features MS \
  --seq_len 36 \
  --label_len 12 \
  --pred_len 24 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 --batch_size 16
""".split()
args = parser.parse_args(argv)

In [5]:
explainer_name_map = {
    "deep_lift":DeepLift,
    "gradient_shap":GradientShap,
    "integrated_gradients":IntegratedGradients,
    "lime":Lime,
    "occlusion":Occlusion,
    # "augmented_occlusion":AugmentedOcclusion,
    "dyna_mask":DynaMask,
    "feature_ablation":FeatureAblation
}

# Initialize Experiment

In [6]:
set_random_seed(args.seed)
# Disable cudnn if using cuda accelerator.
# Please see https://captum.ai/docs/faq#how-can-i-resolve-cudnn-rnn-backward-error-for-rnn-or-lstm-network
# args.use_gpu = False

assert args.task_name == 'long_term_forecast', "Only long_term_forecast is supported for now"
Exp = Exp_Long_Term_Forecast

setting = stringify_setting(args)

In [7]:
exp = Exp(args)  # set experiments
exp.model.load_state_dict(
    torch.load(os.path.join('checkpoints/' + setting, 'checkpoint.pth'))
)
result_folder = './results/' + setting + '/'

Use GPU: cuda:0


# Evaluation

## Model

In [8]:
model = exp.model
model.eval()

# only need to output targets, sinec interpretation is based on outputs
assert not exp.args.output_attention

## Evaluate

In [9]:
flag = 'test'
_, dataloader = exp._get_data(flag)

test 73


In [10]:
expl_metric_map = {
    'mae': mae, 'mse': mse
}
areas = [0.03, 0.05, 0.1, 0.2]

explainers = ['deep_lift', 'feature_ablation'] # explainers = args.explainers
explainers_map = dict()
for name in explainers:
    explainers_map[name] = explainer_name_map[name](model)

In [15]:
results = []
# "zero", "aug", "random"
# performance order random > zero > aug
baseline_mode = "random" 

result_columns = ['batch_index', 'explainer', 'metric', 'area', 'comp', 'suff']

progress_bar = tqdm(
    enumerate(dataloader), total=len(dataloader), disable=False
)
for batch_index, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float().to(exp.device)

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()
    # outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    # baseline must be a scaler or tuple of tensors with same dimension as input
    baselines = get_baseline(batch_x, mode=baseline_mode)
    additional_forward_args = (batch_x_mark, dec_inp, batch_y_mark)

    # get attributions
    for name in explainers:
        explainer = explainers_map[name]
        attr = compute_attr(
            batch_x, baselines, explainer, additional_forward_args, args
        )
    
        # get scores
        for area in areas:
            for metric_name, metric in expl_metric_map.items():
                error_comp = metric(
                    model, inputs=batch_x, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=True
                )
                
                error_suff = metric(
                    model, inputs=batch_x, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=False
                )
           
                result_row = [batch_index, name, metric_name, area, error_comp, error_suff]
                results.append(result_row)

100%|██████████| 5/5 [00:15<00:00,  3.20s/it]


## Output

In [16]:
results_df = pd.DataFrame(results, columns=result_columns)
results_df = results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
# results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results.csv'), index=False)
print(results_df)

           explainer metric  area        comp        suff
0          deep_lift    mae  0.03   22.387512   47.552436
1          deep_lift    mae  0.05   35.472929   34.969124
2          deep_lift    mae  0.10   62.132083   12.359752
3          deep_lift    mae  0.20   80.801300   15.408948
4          deep_lift    mse  0.03   22.275765  100.830368
5          deep_lift    mse  0.05   54.752081   56.914517
6          deep_lift    mse  0.10  165.784731    9.525279
7          deep_lift    mse  0.20  286.879962   17.974950
8   feature_ablation    mae  0.03   22.090495   48.157625
9   feature_ablation    mae  0.05   34.997378   35.562492
10  feature_ablation    mae  0.10   61.651758   12.793189
11  feature_ablation    mae  0.20   79.969749   13.870515
12  feature_ablation    mse  0.03   21.877092  102.763078
13  feature_ablation    mse  0.05   53.530500   58.256120
14  feature_ablation    mse  0.10  163.161661   10.077021
15  feature_ablation    mse  0.20  279.247916   15.032560


# TSR

In [52]:
results = []
baseline_mode = 'random'
result_columns = ['batch_index', 'explainer', 'metric', 'area', 'comp', 'suff']

progress_bar = tqdm(
    enumerate(dataloader), total=len(dataloader), disable=False
)
for batch_index, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float().to(exp.device)

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()
    # outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    inputs = batch_x
    # baseline must be a scaler or tuple of tensors with same dimension as input
    baselines = get_baseline(inputs, mode=baseline_mode)
    additional_forward_args = (batch_x_mark, dec_inp, batch_y_mark)

    for name in explainers:
        explainer = TSRTunnel(explainers_map[name])
        attr = compute_tsr_attr(
            args, explainer, inputs=inputs, 
            sliding_window_shapes=(1,1), 
            strides=1, baselines=baselines,
            additional_forward_args=additional_forward_args
        )

        # get scores
        for area in areas:
            for metric_name, metric in expl_metric_map.items():
                error_comp = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=True
                )
                
                error_suff = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=False
                )
            
                result_row = [batch_index, name, metric_name, area, error_comp, error_suff]
                results.append(result_row)
    gc.collect()

100%|██████████| 5/5 [12:00<00:00, 144.15s/it]


In [53]:
tsr_results_df = pd.DataFrame(results, columns=result_columns)
tsr_results_df = tsr_results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
# tsr_results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results_tsr.csv'), index=False)
print(tsr_results_df)

          explainer metric  area        comp        suff
0  feature_ablation    mae  0.03   18.268983   51.226972
1  feature_ablation    mae  0.05   31.097562   38.407842
2  feature_ablation    mae  0.10   58.866410   13.026361
3  feature_ablation    mae  0.20   69.281813    9.093104
4  feature_ablation    mse  0.03   15.637448  114.791965
5  feature_ablation    mse  0.05   42.280784   66.530092
6  feature_ablation    mse  0.10  146.903009   10.185606
7  feature_ablation    mse  0.20  207.503314    6.183767


# Others

In [ ]:
# temporal_mask = torch.zeros_like(batch_x, dtype=int)
# for t in range(batch_x.shape[1]):
#     temporal_mask[:, t] = t

# explainer = FeatureAblation(model)
# time_score = explainer.attribute(
#     inputs=(batch_x),
#     baselines=(batch_x*0),
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     target=0,
#     feature_mask=temporal_mask
# )
# print(score.shape)